In [1]:
pip install ollama

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install langchain langchain_community chromadb unstructured qdrant_client

Note: you may need to restart the kernel to use updated packages.


Ok, now lets do our RAG

In [3]:
import ollama

In [4]:
from langchain.document_loaders import DirectoryLoader
from langchain_community.document_loaders import UnstructuredMarkdownLoader

It is expected that the docs directory from snappcloud gitlab documentation is copied and available in the path below

In [33]:
loader = DirectoryLoader('./docs',glob="**/*.md",loader_cls=UnstructuredMarkdownLoader)
embeddingModel = 'mxbai-embed-large'
llmModel = 'llama3'
systemPrompt = "You are a helpful assistant that answers questions using the context provided. Cite the relevant documents when you provide an answer"

In [6]:
docs = loader.load()
print(len(docs))

127


We can skip the splitting etc, as the documents are already split, but if you wanted to, you can use the following code

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
print(len(splits))

Now we use our embedding model to create the embeddings and store them in chroma

In [18]:
%%time
from langchain.embeddings import OllamaEmbeddings
embeddings = OllamaEmbeddings(model=embeddingModel)

CPU times: user 74 µs, sys: 4 µs, total: 78 µs
Wall time: 80.1 µs


We should save the embeddings in a vector db, such as chroma or qdrant. Below are both the examples but you can change the one you want to use as code, and the other field can be set as markdown text.

In [35]:
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=docs, embedding=embeddings,persist_directory='./chromadb')

In [36]:
retriever = vectorstore.as_retriever()

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def ollama_llm(question, context):
    formatted_prompt = f"Question: {question}\n\nContext: {context}"
    response = ollama.chat(model=llmModel, options = { 'temperature': 0 }, messages=[{'role': 'system', 'content': systemPrompt},{'role': 'user', 'content': formatted_prompt}])
    return response['message']['content']


In [37]:
# Define the RAG chain
def rag_chain(question):
    retrieved_docs = retriever.invoke(question)
    formatted_context = format_docs(retrieved_docs)
    print("using from context",len(retrieved_docs))
    return ollama_llm(question, formatted_context)

In [38]:
%%time
result = rag_chain("What address should I use for jaeger agent on snappcloud?")
print(result)

using from context 4
Based on the context provided, it seems like you're asking about the Jaeger agent configuration for SnapCloud.

According to the official documentation [1], when using Jaeger with SnapCloud, you should configure the Jaeger agent to use the following address:

`http://localhost:14250/api/traces`

This is because SnapCloud runs its own Jaeger agent internally, and this address allows your application to communicate with it.

Please note that this information is based on the official documentation [1], which provides detailed instructions for configuring Jaeger with various platforms, including SnapCloud.

References:
[1] https://github.com/jaegertracing/jaeger-client-java#snapcloud

Please let me know if you have any further questions or concerns!
CPU times: user 52.7 ms, sys: 4.92 ms, total: 57.6 ms
Wall time: 6.15 s


In [39]:
%%time
result = rag_chain("What are the external IPs for snappcloud that i need to whitelist?")
print(result)

using from context 4
According to the SnapCloud documentation, the external IPs that you need to whitelist are:

* `api.snapcloud.com`
* `cdn.snapcloud.com`
* `cdn2.snapcloud.com`

These IP addresses are used for various services provided by SnapCloud, including caching and proxying. Whitelisting these IPs will ensure that your network allows traffic from SnapCloud's services.

Please note that this information is subject to change, and you should always check the official SnapCloud documentation or contact their support team for the most up-to-date and accurate information.
CPU times: user 8.7 ms, sys: 915 µs, total: 9.61 ms
Wall time: 6.32 s


In [40]:
result = rag_chain("how can I use GPU for my machine learning projects?")
print(result)

using from context 4
To use a GPU for your machine learning projects, you'll need to:

1. **Choose a suitable framework**: Popular options include TensorFlow, PyTorch, and Keras. Each has its own strengths and weaknesses.
2. **Install the necessary libraries**:
	* For TensorFlow: `tensorflow-gpu` (install using pip or conda)
	* For PyTorch: `torch-cuda` (install using pip or conda)
	* For Keras: You'll need to install a backend that supports GPU acceleration, such as TensorFlow or Theano
3. **Verify your GPU is recognized**: Run `nvidia-smi` (for NVIDIA GPUs) or `lspci | grep VGA` (for AMD/Intel GPUs) to ensure your GPU is detected by the system.
4. **Configure your environment**:
	* For TensorFlow: Set the `CUDA_VISIBLE_DEVICES` environment variable to specify which GPU(s) you want to use.
	* For PyTorch: Set the `CUDA_DEVICE_ORDER` and `CUDA_VISIBLE_DEVICES` environment variables to control which GPU(s) are used.
5. **Train your model on the GPU**: Use the framework's built-in suppor